# Twitter data collection

#### References

searchtweets API reference: https://twitterdev.github.io/search-tweets-python/  
Twitter API reference: https://developer.twitter.com/en/docs/tweets/search/api-reference/premium-search.html  
Twitter tweet object and dictionary: https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object

## Imports and credentials

In [25]:
from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results

# general imports
import numpy as np
import pandas as pd
from textblob import TextBlob
import re
import time
import datetime

# plotting and visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

premium_search_args_30day = load_credentials("~/.twitter_keys.yaml",
                                          yaml_key="search_tweets_premium_30day",
                                          env_overwrite=False)
premium_search_args_fullarchive = load_credentials("~/.twitter_keys.yaml",
                                          yaml_key="search_tweets_premium_fullarchive",
                                          env_overwrite=False)

Grabbing bearer token from OAUTH
Grabbing bearer token from OAUTH


## Functions

In [26]:
def days_to_collect(start, end, frequency):
    '''
    will return an array starting at midnight of desired date to last frequency hour of end date
    start = start date
    end = end date
    frequency = number of hours to step by per day. For example frequency = 12, will collect twice: at midnight and noon
    '''
    # add one day for right_side border case
    # pd.date_range only allows dates, use rounding dates and closed='right' to get desired dates
    #print(start, end)
    start = datetime.datetime.strptime(start, '%Y-%m-%d') - datetime.timedelta(days=0, hours=int(frequency))
    end = datetime.datetime.strptime(end, '%Y-%m-%d') + datetime.timedelta(days=1, hours=0)
    #print(start, end)
    dates = pd.date_range(start=start, end=end, freq=frequency+'H', closed='left')
    formatted_dates = [ datetime.datetime.strftime(t, '%Y%m%d%H%M') for t in dates ]
    #print(formatted_dates)
    return formatted_dates

def collect_tweets(from_date, to_date, max_results):
    # maxResults is capped at 100 for sandbox account, even though there should be a next function to get more, it 
    # appears max_results=500 is accepted without any extra work
    # date format: YYYY-mm-DD HH:MM
    # from_date is inclusive. to_date is non-inclusive. Appears to start at from_date and start collecting tweets working
    # backwards to to_date
    bitcoin_rule = gen_rule_payload("bitcoin", results_per_call=100, from_date=from_date, to_date=to_date) 
    print(bitcoin_rule)
    collected_tweets = collect_results(bitcoin_rule, max_results=max_results, result_stream_args=premium_search_args)
    return collected_tweets

## Dates

In [27]:
example_start_date = '2018-10-11'
example_end_date = '2018-10-15'
interval = 24
results_per_call=100
max_results = 100

print("The intra-day hour interval is set to", interval, "Edit the code if desired to change this")
print("The number of tweets per interval is set to", max_results, "Edit the code if desired to change this")
print("please input two dates in the format below to collect dates\n\t", example_start_date, example_end_date, "\n")
user_dates = input("\t")
print()
start_date, end_date = user_dates.split(' ')
test_dates = days_to_collect(start_date, end_date, str(interval))

if (datetime.datetime.fromtimestamp(time.time()) - datetime.datetime.strptime(start_date, '%Y-%m-%d')).days < 30:
    premium_search_args = premium_search_args_30day
    print("will use 30-day dev environment")
else:
    premium_search_args = premium_search_args_fullarchive
    print("will use full-archive dev environment")
    
print("\ntwitter recognized dates will be collected on the closed iterval from", start_date, "to", end_date, "spaced in", str(interval), "hour intervals")

The intra-day hour interval is set to 24 Edit the code if desired to change this
The number of tweets per interval is set to 100 Edit the code if desired to change this
please input two dates in the format below to collect dates
	 2018-10-11 2018-10-15 

	2017-06-04 2017-07-07

will use full-archive dev environment

twitter recognized dates will be collected on the closed iterval from 2017-06-04 to 2017-07-07 spaced in 24 hour intervals


## Twitter call

In [28]:
tweets = []
for i in range(0,len(test_dates[:-1])):
    tweets = np.append(tweets, collect_tweets(test_dates[i], test_dates[i+1], max_results=max_results))
    
    # Requests are limited to 30 per minute for sandbox, 60 for subscriptions 
    # Requests are limited to 10 per second
    num_calls = (i + 1) * max_results//results_per_call
    if num_calls % 2 == 0 and num_calls % 20 != 0:
        print("waiting 10 seconds")
        time.sleep(10)
    if num_calls % 10 == 0:
        print("waiting 60 seconds")
        time.sleep(60)

{"query": "bitcoin", "maxResults": 100, "toDate": "201706040000", "fromDate": "201706030000"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201706050000", "fromDate": "201706040000"}
waiting 10 seconds
{"query": "bitcoin", "maxResults": 100, "toDate": "201706060000", "fromDate": "201706050000"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201706070000", "fromDate": "201706060000"}
waiting 10 seconds
{"query": "bitcoin", "maxResults": 100, "toDate": "201706080000", "fromDate": "201706070000"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201706090000", "fromDate": "201706080000"}
waiting 10 seconds
{"query": "bitcoin", "maxResults": 100, "toDate": "201706100000", "fromDate": "201706090000"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201706110000", "fromDate": "201706100000"}
waiting 10 seconds
{"query": "bitcoin", "maxResults": 100, "toDate": "201706120000", "fromDate": "201706110000"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201706130000", "fromDate": 

retrying request; current status code: 429
retrying request; current status code: 429
retrying request; current status code: 429
HTTP Error code: 429: Request exceeds account’s current package request limits. Please upgrade your package and retry or contact Twitter about enterprise access.
Rule payload: {'query': 'bitcoin', 'maxResults': 100, 'toDate': '201707060000', 'fromDate': '201707050000'}


HTTPError: 

In [14]:
tweets[0:2]

array([{'created_at': 'Tue Aug 15 23:59:59 +0000 2017', 'id': 897608842451849216, 'id_str': '897608842451849216', 'text': 'RT @MarketWatch: Here\'s a chart of the price of bitcoin vs. Google searches for the word "bitcoin." Notice anything? https://t.co/F9sm7vKqc…', 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Mobile Web (M2)</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 705865830660075520, 'id_str': '705865830660075520', 'name': 'KeenMarkets', 'screen_name': 'Keen_Markets', 'location': None, 'url': 'http://keenmarkets.com/resale', 'description': 'Your hub for marketing tips, reviews, and automated software that can assist in many online tasks!', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1383, 'friends_count': 3810, 'listed_count': 162, 'favourites_count': 3266, 'statuses_

## To dataframe and csv

In [29]:
def to_df(tweets):
    # create a pandas df from tweets
    S2 = pd.DataFrame(columns=['tweets', 'date', 'user_name', 'user_screen_name', 'user_followers', 
                           'user_friends', 'user_verified', 'user_language', 'retweet_count', 'favorite_count'])

    for i, tweet in enumerate(tweets):
        S2.loc[i] = [tweet['text'], 
                     tweet['created_at'], 
                     tweet['user']['name'], 
                     tweet['user']['screen_name'], 
                     tweet['user']['followers_count'], 
                     tweet['user']['friends_count'], 
                     tweet['user']['verified'], 
                     tweet['user']['lang'], 
                     tweet['retweet_count'], 
                     tweet['favorite_count']] 
    return S2

S2 = to_df(tweets)

In [30]:
S2.tail()

,tweets,date,user_name,user_screen_name,user_followers,user_friends,user_verified,user_language,retweet_count,favorite_count
3195,RT @AnselLindner: #Bitcoin Core is innovation....,Tue Jul 04 23:55:24 +0000 2017,⚡️urvivor Crypto,CryptoBags021,135,818,False,en,0,0
3196,RT @gilgam_es: Goldman Sachs: Bitcoin Price Is...,Tue Jul 04 23:55:21 +0000 2017,rahmat,prajurit88,1650,806,False,en,0,0
3197,RT @iamorbz: The future of the #IoT #IoE #Bitc...,Tue Jul 04 23:55:20 +0000 2017,J.R Official,harris1nonly,1381,4554,False,en,0,0
3198,RT @RedditBTC: How you can help ensure BIP148 ...,Tue Jul 04 23:55:20 +0000 2017,⚡️urvivor Crypto,CryptoBags021,135,818,False,en,0,0
3199,Our latest podcast about our Bitcoin Beginner ...,Tue Jul 04 23:55:15 +0000 2017,Alex Eaton,alexeaton1411,787,249,False,en,0,0


In [31]:
# save file to csv
'''
S2_tweets = S2.loc[:,['tweets']]
S2_meta = S2.drop(['tweets'], axis=1)

filename = 'complete_tweets/tweets_' + start_date + '_' + end_date
S2_tweets.to_csv(filename + '_Tweets.csv', index=False)
S2_meta.to_csv(filename + '_Metadata.csv', index=False)
print('saved files', filename + '_Tweets.csv', 'and', filename + '_Metadata.csv')
'''

filename = 'complete_tweets/tweets_' + start_date + '_' + end_date + '.csv'
S2.to_csv(filename, index=False)
print('saved file', filename)

saved file complete_tweets/tweets_2017-06-04_2017-07-07.csv
